<h1><center>Laboratorio 3: La desperación de Mr. Cheems 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Otoño 2025</strong></center>

### Cuerpo Docente:

- Profesores: Stefano Schiappacasse, Sebastián Tinoco
- Auxiliares: Melanie Peña, Valentina Rojas
- Ayudantes: Angelo Muñoz, Valentina Zúñiga

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Juan Vicente Onetto Romero
- Nombre de alumno 2: Renzo Zanca


### **Link de repositorio de GitHub:** [Repositorio](https://github.com/RenzoZanca/MDS7202/)

## Temas a tratar
- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines y Column Transformers

## Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: 6 días de plazo con descuento de 1 punto por día. Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda fuertemente asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

### Objetivos principales del laboratorio
- Comprender cómo aplicar pipelines de Scikit-Learn para generar procesos más limpios en Feature Engineering.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [ ]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
except:
    print('Ignorando conexión drive-colab')

# Feature engineering en datos de retail 🛍️

### 0. Cargar Dataset

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

Mr. Cheems, gerente de una cotizada tienda de retail en Europa, les solicita si pueden analizar los datos de algunas de sus tiendas. En una reunión, Mr Cheems le comenta que la calidad de sus datos no es muy buena, por lo que le solicita a usted que limpie su base de datos y cree nuevos atributos relevantes para el negocio.

Por ello, el área de ventas les entrega archivo llamado `online_retail_data.pickle` el cual usted decide cargar a continuación.

In [ ]:
# Inserte su código aquí
path = 'drive/MyDrive/online_retail_data.pickle'
df_retail = pd.read_pickle(path)
df_retail.head()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### 1. Función para explorar características [0.5 puntos]

<p align="center">
  <img width=300 src="https://editor.analyticsvidhya.com/uploads/47389meme.png">
</p>




Tras inspeccionar brevemente los datos proporcionados, usted decide crear una función que realice lo siguiente:
- Plotee un histograma para las variables precios y cantidad. [0.3 puntos]
- Imprima un conteo de datos nulos por variable [0.2 puntos]

**Nota**: Para generar los gráficos no es obligatorio el uso de `plotly`, pero si es altamente recomendado. Pueden encontrar más información de esta librería en este [enlace](https://plotly.com/python/).

**Respuesta:**

In [ ]:
def explore_data(dataframe_in):
    # Plot histogramas para Price y Quantity
    fig = px.histogram(dataframe_in, x=['Price', 'Quantity'],
                       nbins=50, title='Distribución de Precio y Cantidad',
                       labels={'value': 'Valor', 'variable': 'Variable'},
                       marginal='box')
    fig.show()

    # Conteo de nulos por columna
    null_counts = dataframe_in.isnull().sum()
    print("Conteo de valores nulos por columna:")
    print(null_counts)


explore_data(df_retail)


### 2. Eliminando outliers [1.0 puntos]

<p align="center">
  <img width=300 src="https://media.licdn.com/dms/image/C5612AQGdXKCka7HumA/article-cover_image-shrink_600_2000/0/1520056407281?e=2147483647&v=beta&t=VZcfjjzjK4LxXdZkSu1KisWC0Ry8bk4tPCn3R8aYdNM">
</p>




#### 2.1 Creando la clase IQR [0.5 puntos]

Entre las falencias de los datos, Mr. Cheems le comenta que a veces los operadores no ingresan el precio correcto de los productos. Mr. Cheems le comenta que se dio cuenta de este fenómeno porque hay productos con precios exagerádamente altos o bajos. Por lo cual usted decide eliminar outliers del dataframe a traves del rango intercuartil el cual cuenta con los siguientes pasos:

1. Calcular el primer cuartil $Q1$ y el tercer cuartil $Q3$. Hint: utilice el método `quantile()`

2. Calcular el rango intercuartil (RIC): $RIC = Q3 - Q1$

3. Calcular los límites para identificar outliers:
 - Límite inferior: $~~Q1 - \lambda \cdot RIC$
 - Límite superior: $~~Q3 + \lambda \cdot RIC$

4. Eliminar outliers: Los outliers son los datos que están por debajo del límite inferior o por encima del límite superior.


Para realizar dicha tarea, usted decide crear una clase llamada `IQR()` utilizando `BaseEstimator` y `TransformerMixin` para realizar una transformación de cada una de las columnas numéricas del DataFrame utilizando `ColumnTransformer()` más tarde. Considere que lambda debe ser $\lambda$ un parámetro a definir por el usuario.

**Hint:** tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer).

**Nota:** No modificar el método set_output de la clase IQR

**Respuesta:**

In [ ]:
class IQR(BaseEstimator, TransformerMixin):
    def __init__(self, lambda_value=1.5):
        self.lambda_value = lambda_value

    def fit(self, X, y=None):
        X = pd.DataFrame(X)
        q1 = X.quantile(0.25)
        q3 = X.quantile(0.75)
        iqr = q3 - q1
        # store the limits:
        self.lower_limits_ = q1 - self.lambda_value * iqr
        self.upper_limits_ = q3 + self.lambda_value * iqr
        return self

    def transform(self, X):
        X = pd.DataFrame(X)
        X_transformed = X.copy()
        for col in X.columns:
            if col in self.lower_limits_:
                lower = self.lower_limits_[col]
                upper = self.upper_limits_[col]
                X_transformed[col] = X[col].clip(lower, upper)
        return X_transformed

    def set_output(self, transform=None):
        return self


#### 2.2 Creación del Pipeline [0.5 puntos]

Para comenzar introduciéndose en el uso de pipeline, usted decide definir un pipeline con el Transformer previamente definido. Además, usted decide visualizar cómo cambia la distribución de las variables Precio y Cantidad antes y despues de aplicar IQR. Para ello, usted aplica los siguientes pasos:

- Definir un pipeline llamado `numeric_transformations` para las variables precio y cantidad con la transformación IQR. [0.1 puntos]
- Defina un column transformer que aplique `numeric_transformations` para las variables numéricas y `passthrough` para las variables categóricas. Adicionalmente, fije el parámetro `verbose_feature_names_out` en `False`. Ver hint al final [0.1 puntos]
- Defina el dataframe `df_iqr` aplicado el column transformer a los datos proporcionados por Mr. Cheems considerando un valor de $\lambda$ que tenga un desempeño aceptable para ambas variables. [0.1 puntos]
- Usar `explore_data` en `df_retail` y en `df_iqr`.  [0.1 puntos]
- Reportar los cambios observados en la distribución de las variables. ¿Qué sucede al aumentar el valor de lambda? [0.1 puntos]


**Hint:** El transformador `passthrough` está predefinido y es una opción que puedes usar para las columnas que no deseas transformar. Al especificar 'passthrough' para una parte de tu ColumnTransformer, las columnas correspondientes pasarán a través del ColumnTransformer sin ninguna modificación. El siguiente [enlace](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) le puede ser útil.

**Nota:** Mantenga el método set_output del column transformer con la transformación `pandas` para obtener un dataframe una vez aplicado el column transformer.

**Respuesta:**

Apóyese de la siguiente estructura para su respuesta:

In [ ]:
# Definicion las variables que pasarán por cada pipeline
numerical_columns = ['Price', 'Quantity']
categorical_columns = ['Invoice', 'StockCode', 'Description', 'Customer ID', 'Country', 'InvoiceDate']

# Definicion del pipeline
numeric_transformations = Pipeline([
    ('iqr', IQR(lambda_value=1.5))
])

# ColumnTransformer
column_transformer = ColumnTransformer([
    ('numerical', numeric_transformations, numerical_columns),
    ('categorical', 'passthrough', categorical_columns)
], verbose_feature_names_out=False)

column_transformer.set_output(transform='pandas')

# Aplicamos ColumnTransformer a los datos
df_iqr = column_transformer.fit_transform(df_retail)

# Gráficos
print("Datos originales:")
explore_data(df_retail)

print("\nDatos después de IQR:")
explore_data(df_iqr)

Podemos observar que debido a los outliers, la data original se concentra casi toda en valores más pequeños, con solo algunos datos con valores superiores. Por el contrario, al aplicar IQR se eliminan los outliers y los datos se distribuyen de manera más uniforme sobre el eje x. Por otro lado tenemos que mientras mayor es el valor de lambda, elimina menor cantidad de outliers, pues por la fórmula, se obtiene un rango más grande para considerar valores como outliers.

### 3. Agregando un imputer al pipeline [1.0 puntos]



<p align="center">
  <img width=300 src="https://media.makeameme.org/created/hmm-there-is.jpg">
</p>

Para continuar con la limpieza del dataframe usted decide imputar los datos nulos de las variables numéricas, para lo cual decide realizar las siguientes tareas:

1. Crear un pipeline para variables categóricas llamado `categoric_transformations` con un paso llamado `mode_imputer`, en el cual se imputen los datos faltantes por la categoría más frecuente.
2. Agregar al pipeline `numeric_transformations` un paso llamado `mean_imputer`, en el cual se imputen los datos por la media usando [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) [0.1 puntos]
3. Crear y aplicar un `ColumnTransformer` actualizado con los pipelines `categoric_transformations` y `numeric_transformations` a `df_retail`, creando un dataframe llamado `df_mean_imputer`. [0.1 puntos]
4. Comparar los resultados de `explore_data` en `df_mean_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [0.2 puntos]
5. Cambiar el imputer de `numeric_transformations` por [KNNImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html) y definir un nuevo dataframe llamado `df_knn_imputer`, aplicando el nuevo ColumnTransformer a `df_retail`. En caso de los tiempos de ejecución sean altos puede probar a reducir el parámetro `n_neighbors`. [0.1 puntos]
6. Comparar los resultados de `explore_data` en `df_knn_imputer` y `df_iqr`. ¿Qué diferencias observa en la distribución de los datos? [0.2 puntos]
7. Comparar los resultados de `explore_data` en `df_knn_imputer` y `df_mean_imputer`. ¿Cuál método de imputación es mejor? Deje el método escogido en el ColumnTransformer. [0.2 puntos]

**Nota: Fije el parámetro verbose_feature_names_out en `False` y utilice el método set_output con transformación `pandas` en cada ColumnTransformer para obtener como salida un dataframe.**

**Respuesta:**

In [ ]:
from sklearn.impute import SimpleImputer, KNNImputer

# Pipeline para variables categóricas
categoric_transformations = Pipeline([
    ('mode_imputer', SimpleImputer(strategy='most_frequent'))
])

# Pipeline para variables numéricas con imputación por media
numeric_transformations_mean = Pipeline([
    ('mean_imputer', SimpleImputer(strategy='mean')),
    ('iqr', IQR(lambda_value=1.5))
])

# ColumnTransformer con imputación por media
column_transformer_mean = ColumnTransformer([
    ('numerical', numeric_transformations_mean, numerical_columns),
    ('categorical', categoric_transformations, categorical_columns)
], verbose_feature_names_out=False)

column_transformer_mean.set_output(transform='pandas')
df_mean_imputer = column_transformer_mean.fit_transform(df_retail)

# Comparaciones
print("\nDatos con imputación por media:")
explore_data(df_mean_imputer)

print("\nDatos con solo iqr")
explore_data(df_iqr)

4: Notamos que la distribución de los datos se mantiene más o menos similar, cambiando unicamente que con la imputación se agregan más valores en el promedio de las variables.

In [ ]:

# Pipeline para variables numéricas con imputación KNN
numeric_transformations_knn = Pipeline([
    ('knn_imputer', KNNImputer(n_neighbors=5)),
    ('iqr', IQR(lambda_value=1.5))
])

# ColumnTransformer con imputación KNN
column_transformer_knn = ColumnTransformer([
    ('numerical', numeric_transformations_knn, numerical_columns),
    ('categorical', categoric_transformations, categorical_columns)
], verbose_feature_names_out=False)

column_transformer_knn.set_output(transform='pandas')
df_knn_imputer = column_transformer_knn.fit_transform(df_retail)

print("\nDatos con imputación KNN:")
explore_data(df_knn_imputer)

print("\nDatos con solo iqr")
explore_data(df_iqr)

6: Observamos que la distribución es más o menos similar entre ambos dataframes. Esto se debe al funcionamiento de KNN el cual reparte los nulos a 5 puntos del datasets según cual está más cercano, manteniendo más la distribución que asignarlo solo a un punto.

/: Notamos que para la imputación con KNN no se concentran muchos valores en un único lugar, haciendolo que los datos estén más distribuidos y no tan concentrados en la media. Con esto en mente, elegimos KNN ya que mantiene más la distribución de los datos, pero eliminando los valores nulos.

### 4. Creación de nuevas features [2.0 puntos]

<p align="center">
  <img width=250 src="https://miro.medium.com/max/1000/1*JtTWgAcfVTWV8OTjT47Atg.jpeg">
</p>


#### 4.1 Definicion de LRMFP [1.0 puntos]

Dado que Mr. Lepin está interesado en obtener nuevos atributos relevantes para su negocio, su equipo de expertos sugiere la construcción de variables **LRMFP**, las que se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales. **Considere "hoy" como la fecha mas reciente del dataset**.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente.

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.


$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**<u>Formato</u> del Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [ ]:
def custom_features(dataframe_in):
    dataframe_in = dataframe_in.copy()
    # calculamos el precio total
    dataframe_in['TotalPrice'] = dataframe_in['Quantity'] * dataframe_in['Price']
    today = dataframe_in['InvoiceDate'].max()

    # agrupamos por cliente y fecha de compra
    visits = dataframe_in.groupby(['Customer ID', 'InvoiceDate'], observed=True)['TotalPrice'].sum().reset_index()

    # L: Length = max - min fechas
    L = visits.groupby('Customer ID', observed=True)['InvoiceDate'].agg(lambda x: (x.max() - x.min()).days).rename('L')

    # R: Recency = hoy - fecha de la última compra
    R = visits.groupby('Customer ID', observed=True)['InvoiceDate'].agg(lambda x: (today - x.max()).days).rename('R')

    # M: Monetary = promedio gasto por visita
    M = visits.groupby('Customer ID', observed=True)['TotalPrice'].mean().rename('M')

    # F: Frequency = número de visitas
    F = visits.groupby('Customer ID', observed=True).size().rename('F')

    # P: Periodicity = std(IVT_i)
    ivt = visits.sort_values(['Customer ID', 'InvoiceDate']) # ordenamos por cliente y fecha
    ivt['PrevDate'] = ivt.groupby('Customer ID', observed=True)['InvoiceDate'].shift(1) # obtenemos la fecha de la visita anterior
    ivt['IVT'] = (ivt['InvoiceDate'] - ivt['PrevDate']).dt.days # calculamos el intervalo de tiempo entre visitas
    ivt['IVT'] = ivt['IVT'].fillna(0) # rellenamos con 0 si no hay intervalo de tiempo
    P = ivt.groupby('Customer ID', observed=True)['IVT'].std().fillna(0).rename('P') # calculamos la desviación estándar de los intervalos de tiempo entre visitas

    # Unir todo
    dataframe_out = pd.concat([L, R, M, F, P], axis=1).reset_index()

    return dataframe_out

#### 4.2 Agregando las custom features [1.0 puntos]

Ahora, usted decide agregar al pipeline las nuevas variables creadas, para lo cual realiza las siguientes tareas:

1. Cree un nuevo pipeline llamado `retail_pipeline` que encapsule el ColumnTransformer y calcule las LRMFP. El primer paso del pipeline llámelo  `col_tranformer` y el segundo paso llámelo `custom_features`, incorpora las nuevas variables al dataframe. Hint: les puede ser útil investigar [este](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html) método. [0.1 puntos]
2. Aplicar el pipeline actualizado a los datos proporcionados por Mr. Cheems, creando un nuevo dataframe llamado `df_custom`. [0.1 puntos]
3. Explorar la distribución de las nuevas variables con `explore_data` y comentar brevemente (2-3 líneas) características de cada custom feature. [0.5 puntos]
5. Entregar un insight para el negocio en base a las nuevas variables. [0.3 puntos]

**Nota:** Recuerde fijar el parámetro `verbose_feature_names_out` en `False` e incorporar el método `set_output` para obtener una salida en formato dataframe del ColumnTransformer.

In [ ]:
print(df_retail.columns)


**Respuesta**

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

#Creamos un custom_transformer que use FunctionTransfomer para aplicar la función custom_features
custom_transformer = FunctionTransformer(custom_features)


retail_pipeline = Pipeline([
    ('col_transformer', column_transformer),  # Esto es el ColumnTransformer definido previamente
    ('custom_features', custom_transformer)     # Aplica la función custom_features
])

df_custom = retail_pipeline.fit_transform(df_retail) #Se aplica el pipeline y se guarda en un nuevo DF




In [ ]:
def explore_data_custom(dataframe_in):
    # Para todas las columnas excepto 'Customer ID', asumiendo que es un identificador
    cols = [col for col in dataframe_in.columns if col != 'Customer ID']
    fig = px.histogram(dataframe_in.melt(id_vars='Customer ID', value_vars=cols),
                       x='value', color='variable',
                       nbins=50,
                       title='Distribución de custom features (LRMFP)',
                       marginal='box')
    fig.show()
    print("Estadísticas descriptivas:")
    print(dataframe_in[cols].describe())

explore_data_custom(df_custom)


### 4.2.3


L:
*   Mediana: 105 días, Media: 134 días.
* La mayoría de los clientes tienen relaciones de compra que duran entre 3 y 4 meses.
* Algunos clientes muestran relaciones prolongadas (hasta 373 días).

R:
* Mediana: 52 días, Media: 90 días.
* En promedio, los clientes realizaron su última compra hace aproximadamente 2 meses.
* Existen clientes que llevan hasta 373 días sin comprar, señalando oportunidades de reactivación.

M:
* Mediana: ~228, Media: ~277.
* Indica un gasto moderado por visita en promedio.
* Algunos clientes alcanzan gastos muy elevados (cercanos a 3900), reflejando una gran dispersión y la presencia de clientes de alto valor.

F:
* Mediana: 2 visitas, Media: ~4.44 visitas.
* La mayoría de los clientes realizan pocas compras.
* Un grupo pequeño de clientes tiene una frecuencia muy alta (máximo de 202 visitas).

P:
* Mediana: ~22 días, Media: ~37 días.
* La mayoría de los clientes presentan intervalos de compra relativamente regulares.
* Existe una alta variabilidad en algunos casos (valores hasta 255 días), indicando patrones de compra irregulares.




### 4.2.4


* Clientes de Alto Valor y Frecuencia:

  Los que presentan altos valores en Frequency (F) y Monetary (M) son clientes leales y de alto valor.

  Son buenos candidatos para programas VIP o recompensas exclusivas, ya que generan un ingreso considerable de forma recurrente.


* Oportunidades de Reactivación:

  Clientes con altos valores en Recency (R) indican inactividad desde hace meses. Podría ser conveniente realizar campañas para "reactivarlos".

* Periodos de compra regulares:
  
  Podría ser interesante generar campañas de marketing preeviendo la periodicidad "regular" de los clientes.

### 5. MinMax Scaler [1.0 puntos]

<p align="center">
  <img width=300 src="https://i.imgflip.com/1fsprn.jpg">
</p>


#### 5.1 Definición del Column Transformer [0.5 puntos]

Construya una clase llamada `MinMax()` para realizar una transformación de cada una de las columnas de un DataFrame utilizando `ColumnTransformer()`. Recuerde  usar `BaseEstimator` y `TransformerMixin`.


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$


Consulte el siguiente [link](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer) si tiene dudas sobre la creación de custom transformers.

**Respuesta:**

In [ ]:
class MinMax(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        # Convertimos X a DataFrame para asegurarnos de tener los métodos de Pandas
        X = pd.DataFrame(X)
        # Calculamos el mínimo y máximo de cada columna
        self.min_ = X.min()
        self.max_ = X.max()
        return self

    def transform(self,X):
        # Convertimos a DataFrame y hacemos un copy
        X = pd.DataFrame(X).copy()
        # Aplicamos la transformación MinMax
        X_transformed = (X - self.min_) / (self.max_ - self.min_)
        return X_transformed

    def set_output(self,transform='default'):
        #No modificar este método
        return self



#### 5.2 Incorporando MinMax al pipeline [0.5 puntos]

Ahora, usted decide agregar el escalamiento al pipeline, para lo que decide seguir los siguientes pasos:

- Agregar el paso `minmax` al pipeline `numeric_transformations`, haciendo uso de la clase creada. [0.1 puntos]
- Defina el dataframe `df_minmax` aplicando el ColumnTransformer actualizado a los datos proporcionados por Mr. Cheems. [0.1 puntos]
- Usar `explore_data` en `df_retail` y en `df_minmax`. [0.1 puntos]
- Reportar los cambios observados en la distribución de las variables.  [0.2 puntos]

**Nota:** Recuerde fijar el parámetro `verbose_feature_names_out` en `False` e incorporar el método `set_output` para obtener una salida en formato dataframe del ColumnTransformer.

**Respuesta:**

In [ ]:
numerical_columns = ['Price', 'Quantity']
categorical_columns = ['Invoice', 'StockCode', 'Description', 'Customer ID', 'Country', 'InvoiceDate']

# Definicion del pipeline

# Pipeline para variables categóricas
categoric_transformations = Pipeline([
    ('mode_imputer', SimpleImputer(strategy='most_frequent'))
])


numeric_transformations = Pipeline([
    ('mean_imputer', SimpleImputer(strategy='mean')),
    ('iqr', IQR(lambda_value=1.5)),
    ('minmax', MinMax())
])

# ColumnTransformer

column_transformer = ColumnTransformer([
    ('numerical', numeric_transformations, numerical_columns),
    ('categorical', categoric_transformations, categorical_columns)
], verbose_feature_names_out=False)

column_transformer.set_output(transform='pandas')

# Nuevo df
df_minmax= column_transformer.fit_transform(df_retail)

# Gráficos
print("Datos originales:")
explore_data(df_retail)

print("\nDatos después de MinMax:")
explore_data(df_minmax)




El pipeline combina imputación por la media y recorte de outliers para luego aplicar MinMax, lo que normaliza las variables "Price" y "Quantity" en el rango [0,1]. Esto reduce la influencia de valores extremos  y concentra la mayoría de los datos en rangos bajos. Esto consolida los datos en una escala uniforme.

### 6. Pregunta teórica [0.5 puntos]

<p align="center">
  <img width=300 src="https://file.coinexstatic.com/2023-09-19/166BAC031F222E5910954E7D7D0BC844.png">
</p>

**Finalmente**, explíquele a Mr. Cheems porqué es útil la creación de pipelines al momento de hacer Feature Engineering en Machine Learning.

**Respuesta:**

Los pipelines aseguran que todas las transformaciones se determinen únicamente a partir de los datos de entrenamiento. No se extrae información del conjunto de prueba, lo que evita el data leakage. Además, los pipelines permiten encapsular todo el pre-procesamiento de los datos, lo que facilita la modificación, integración y testeo.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por el foro de U-cursos o por correo.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>